In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import OPTICS
from kneed import KneeLocator
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA

In [ ]:
df = pd.read_csv("log2.csv")

In [ ]:
test = df.to_numpy()
disease_state = test[0][1:]
genename = test[:,0][1:]
exp = test[1:,1:]

In [ ]:
exp.shape

In [ ]:
data = exp[:,disease_state==1]

In [ ]:
data.shape

In [ ]:
# Select appropriate number of clusters of K-means

kmeans_kwargs = {
	"init": "random",
	"n_init": 10,
	"max_iter": 300,
}

# sse
sse = []
for k in range(1, 16):
	kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
	kmeans.fit(data)
	sse.append(kmeans.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(15,10), dpi=500)
plt.plot(range(1, 16), sse)
plt.xticks(range(1, 16))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.savefig("sse.png")
plt.close()

In [ ]:
kl = KneeLocator(range(1,16), sse, curve="convex", direction="decreasing")
kl.elbow

In [ ]:
silhouette_coefficients = []

for k in range(2, 16):
   kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
   kmeans.fit(data)
   score = silhouette_score(data, kmeans.labels_)
   silhouette_coefficients.append(score)

In [ ]:
silhouette_coefficients

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(15,10), dpi=500)
plt.plot(range(2, 16), silhouette_coefficients)
plt.xticks(range(2, 16))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.savefig("silhouette.png")
plt.close()

In [ ]:
k = 7
model = KMeans(n_clusters=k, random_state=0).fit(data)

In [ ]:
for i in range(k):
	print(sum(model.labels_==i))

In [ ]:
pca = PCA(2)
exp_trans = pca.fit_transform(data)

In [ ]:
sns.scatterplot(exp_trans[:,0], exp_trans[:,1], hue=model.labels_, legend=False)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.savefig("PCA_res.png", dpi=500)


In [ ]:
c_dict = []
for i in range(k):
	tmp = data[model.labels_==i,:]
	tmp_corr = np.corrcoef(tmp.astype(float))
	c_dict.append(tmp_corr)

In [ ]:
c_sum = []
for i in range(k):
	tmp = c_dict[i]
	total = np.sum(tmp, axis=0)
	print(total.shape)
	c_sum.append(total)
	plt.figure(figsize=(15,10), dpi=500)
	plt.scatter([i for i in range(len(total))], total, s=5)
	name = "cluster_" + str(i) + ".png"
	plt.savefig(name)
	plt.close()

In [ ]:
def hubgeneboundary(array):
	sorted_array = array[np.argsort(array)]
	n = len(array) // 10
	return sorted_array[-n]

In [ ]:
hub = []
for i in range(k):
	index = np.where(model.labels_==i)
	index = index[0]
	print(index)
	coff = c_sum[i]
	hub.append(index[coff>=hubgeneboundary(coff)])

In [ ]:
gene = []
for list in range(len(hub)):
	tmp = hub[list]
	for i in range(len(tmp)):
		e = tmp[i]
		gene.append(e)

In [ ]:
out = np.sort(gene)
print(out)
print(genename[0:30])

In [ ]:
res = genename[out]
res = pd.DataFrame(res)
res.to_csv("cluster_gene_k7.csv")

In [ ]:
optics = OPTICS(min_samples=10).fit(exp_trans)

In [ ]:
for i in range(len(np.unique(optics.labels_))):

	print(sum(optics.labels_==(i-1)))